In [9]:
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
from IPython.display import clear_output
from selenium import webdriver
from time import sleep

In [78]:
atm = pd.read_csv('train.csv',index_col=0)

In [79]:
atm.head()

,id,atm_group,address,address_rus,lat,long,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [80]:
atm['street'] = atm.address.apply(lambda x: x[:23].strip())

atm['city'] = atm.address.apply(lambda x: x[23:].strip()).apply(lambda x: x[:-2] if x.endswith(' G') else x)

In [82]:
def transform(x):
    dictionary = {"MOSKVA": "MOSCOW",
                  "SANKT-PETERBU": "SAINT PETERSBURG",
                  'S.-PETERBURG':  "SAINT PETERSBURG",
                  'ST-PETERSBURG': "SAINT PETERSBURG",
                  'ST PETERSBURG': "SAINT PETERSBURG"}
    for key, value in dictionary.items():
        x = x.replace(key, value)
    return x

In [83]:
atm.city = atm.city.map(transform)

In [84]:
to_replace = {
     'AV.': ' PROSPECT ',
     'AVE': ' PROSPECT ', 
     'PR-KT': ' PROSPECT ',
     'PR-T': ' PROSPECT ',
     'STR.': ' ULITSA ',
     ' ST ': ' ULITSA ',
     'UL.' : ' ULITSA ',
     'H/W.': ' SHOSSE ',
     ', LIT. ': '',
     'D.': '',
     'VLAD. ':'',
     ', KORP. ': '/',
     ' DZERZHINSK':  ' DZERZHINSKOGO ',
     '.': '. ',
     ',': ', ',
}

for i in atm.index:
    for key, value in to_replace.items():
        atm.at[i,'street'] =  atm.at[i,'street'].replace(key, value)

for i in atm.index:
    atm.at[i,'street'] = atm.at[i,'street'][4:] if atm.at[i,'street'][:4] == 'ABB ' else atm.at[i,'street']

In [85]:
def parsing(atm, column='info', start=0):
    driver = webdriver.Chrome('chromedriver.exe')
    atm[column] = np.nan
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i, atmi in log_progress(list(atm.iterrows())[start:]):
        urli = url + ' '.join([atmi.address, atmi.city]).replace(' ', '%20')
        driver.get(urli)
        try: 
            element = driver.find_element_by_xpath("//div[@class='card-title-view']")
            atm.loc[i, column] = element.text
        except:
            pass
        sleep(0.2)
    return atm.copy()

In [86]:
atm_new = parsing(atm[4:7])

0


In [87]:
atm_new

,id,atm_group,address,address_rus,lat,long,target,street,city,info
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,"GVARDEYSKAYA PL. , 2",NORILSK,"Гвардейская площадь, 2\nНорильск, Красноярский..."
5,313.0,32.0,"RUSSKAYA, 16 VLADIVOSTOK","Русская улица, 16, Владивосток, Приморский кра...",43.165860,131.908212,-0.145001,"RUSSKAYA, 16",VLADIVOSTOK,"Русская улица, 16\nВладивосток, Приморский кра..."
6,335.0,32.0,"SHUKHOVA, 14 MOSCOW","улица Шухова, 14, Москва, Россия, 115162",55.716859,37.613063,-0.024682,"SHUKHOVA, 14",MOSCOW,"улица Шухова, 14\nМосква, Россия, 115162\n55.7..."
